In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install gspread==3.6


In [3]:


from google.colab import auth
auth.authenticate_user()

import gspread 
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1F8A5n3WVr9WXHDXZhv2w0BPaMC8uymUyxEWdI9UX6xg/edit#gid=0')
sheet = wb.worksheet('results')

def get_next_row():
  current_max_row = len(sheet.get_all_values())
  new_row_to_append = int(current_max_row) + 1
  return f"A{new_row_to_append}"

row = get_next_row()
sheet.update(row, [['blah', 'Lee']])
current_max_row = max(sheet.get_all_records()[0].keys())

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
# pip install df2gspread

In [6]:

# from df2gspread import df2gspread as d2g
# import pandas as pd
# d = [pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
#     pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])]
# df = pd.DataFrame(d)

# # use full path to spreadsheet file
# spreadsheet = '/some/folder/New Spreadsheet'
# # or spreadsheet file id
# # spreadsheet = '1cIOgi90...'

# wks_name = 'New Sheet'

# d2g.upload(df, spreadsheet, wks_name)
# # if spreadsheet already exists, all data of provided worksheet(or first as default)
# # will be replaced with data of given DataFrame, make sure that this is what you need!

In [7]:

# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [8]:
from google.colab import auth
auth.authenticate_user()

import gspread 
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1F8A5n3WVr9WXHDXZhv2w0BPaMC8uymUyxEWdI9UX6xg/edit#gid=0')
sheet = wb.worksheet('results')
sheet.update('A1', [[1, 2], [3, 4]])


{'spreadsheetId': '1F8A5n3WVr9WXHDXZhv2w0BPaMC8uymUyxEWdI9UX6xg',
 'updatedCells': 4,
 'updatedColumns': 2,
 'updatedRange': 'results!A1:B2',
 'updatedRows': 2}

In [12]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [13]:
# !sh setup.sh

In [14]:

!ls

adc.json  drive  sample_data  setup.sh


In [15]:

# import os
# if not os.path.isdir('drive'):
#   os.mkdir('drive')
# os.chdir('drive')

In [16]:
%cd drive/My Drive/mixout/

/content/drive/My Drive/mixout


In [17]:

!pwd

/content/drive/My Drive/mixout


In [18]:
!git clone https://github.com/leedtan/ModernML_TinyBert.git

fatal: destination path 'ModernML_TinyBert' already exists and is not an empty directory.


In [23]:

import os
os.listdir()

['ModernML_TinyBert']

In [24]:
os.chdir('ModernML_TinyBert')
!python download_glue_data.py

	Completed!
	Completed!
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [25]:
# import gspread
# gc = gspread.service_account()

In [26]:

os.chdir('revisit-bert-finetuning')
os.listdir()

['LICENSE',
 'model_utils.py',
 'prior_wd_optim.py',
 'repo_illustration.png',
 'requirements.txt',
 'sample_commands',
 '__init__.py',
 'emily_run_glue_experiment.sh',
 'finetuning_torchvision_models_tutorial.ipynb',
 'mixout-fan-in.ipynb',
 'mixout.ipynb',
 'run_glue_datasets.py',
 'run_glue_experiment.py',
 'ryan.sh',
 'ryan_run_glue_experiment.sh',
 'summarize_results.py',
 '__pycache__',
 'README.md',
 'mixout.py',
 'run_glue.py',
 'options.py']

In [27]:
!pip install transformers==2.8.0

In [28]:
!git checkout some_frozen_mixout_reg_scaling

M	download_wiki_dump.sh
M	extract_and_clean_wiki_dump.sh
M	revisit-bert-finetuning/emily_run_glue_experiment.sh
Already on 'some_frozen_mixout_reg_scaling'
Your branch is up to date with 'origin/some_frozen_mixout_reg_scaling'.


In [29]:
!ls

emily_run_glue_experiment.sh		      README.md
finetuning_torchvision_models_tutorial.ipynb  repo_illustration.png
__init__.py				      requirements.txt
LICENSE					      run_glue_datasets.py
mixout-fan-in.ipynb			      run_glue_experiment.py
mixout.ipynb				      run_glue.py
mixout.py				      ryan_run_glue_experiment.sh
model_utils.py				      ryan.sh
options.py				      sample_commands
prior_wd_optim.py			      summarize_results.py
__pycache__


In [30]:
!cat ryan.sh

../../revisit-bert-finetuning/bert/bin/python run_glue_datasets.py \
    --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
    --do_train --data_dir ../../revisit-bert-finetuning/glue_data --max_seq_length 64 \
    --per_gpu_eval_batch_size 8 --weight_decay 0 --seed 1 \
    --overwrite_output_dir --do_lower_case --per_gpu_train_batch_size 8 \
    --gradient_accumulation_steps 4 --logging_steps 0 --num_loggings 10 \
    --save_steps 0 --test_val_split --use_torch_adamw --cache_dir /home/ubuntu/hf-transformers-cache \
    --num_train_epochs 3.0 --warmup_ratio 0.1 --learning_rate 2e-05 \
    --output_dir tests/FULLTESTS/decay_50_to_20 \
    --reinit_pooler --normalize --mixout_layers 12 --mixout .2 \
    --trials 20 --mixout_decay 0.5

# ../../revisit-bert-finetuning/bert/bin/python run_glue_datasets.py \
#     --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
#     --do_train --data_dir ../../revisit-bert-finetuning/glue_data --max_seq_

In [31]:
!git pull

remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 14 (delta 9), reused 14 (delta 9), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/leedtan/ModernML_TinyBert
   9f7214b..4db0781  some_frozen_mixout_reg_scaling -> origin/some_frozen_mixout_reg_scaling
Updating 9f7214b..4db0781
Fast-forward
 revisit-bert-finetuning/glue_utils.py        | 225 +++++++
 revisit-bert-finetuning/manipulate_model.py  | 122 ++++
 revisit-bert-finetuning/options.py           |  27 +-
 revisit-bert-finetuning/run_glue.py          | 893 +--------------------------
 revisit-bert-finetuning/run_glue_datasets.py |  31 +-
 revisit-bert-finetuning/train.py             | 501 +++++++++++++++
 revisit-bert-finetuning/validate.py          | 125 ++++
 7 files changed, 1047 insertions(+), 877 deletions(-)
 create mode 100644 revisit-bert-finetuning/glue_utils.py
 create mode 100644 revisit-bert-finetu

In [35]:
# !echo '''python run_glue_datasets.py \
#     --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
#     --do_train --data_dir /content/ModernML_TinyBert/glue_data/RTE --max_seq_length 16 \
#     --per_gpu_eval_batch_size 64 --weight_decay 0 --seed 0 \
#     --overwrite_output_dir --do_lower_case --per_gpu_train_batch_size 32 \
#     --gradient_accumulation_steps 1 --logging_steps 0 --num_loggings 10 \
#     --save_steps 0 --test_val_split --use_torch_adamw --cache_dir /content/ModernML_TinyBert/hf-transformers-cache \
#     --num_train_epochs 3.0 --warmup_ratio 0.1 --learning_rate 2e-05 \
#     --output_dir bert_output/REINIT5/RTE/SEED0 \
#     --reinit_pooler --reinit_layers 5''' > sample_commands/run.sh

!echo '''python run_glue_datasets.py \
    --model_type bert --model_name_or_path bert-large-uncased --task_name RTE \
    --do_train --data_dir "/content/drive/My Drive/mixout/ModernML_TinyBert/glue_data" --max_seq_length 64 \
    --per_gpu_eval_batch_size 8 --weight_decay 0 --seed 1 \
    --overwrite_output_dir --do_lower_case --per_gpu_train_batch_size 8 \
    --gradient_accumulation_steps 4 --logging_steps 0 --num_loggings 10 \
    --save_steps 0 --test_val_split --use_torch_adamw --cache_dir "/content/drive/My Drive/mixout/ModernML_TinyBert/hf-transformers-cache" \
    --num_train_epochs 3.0 --warmup_ratio 0.1 --learning_rate 2e-05 \
    --output_dir tests/FULLTESTS/classic --all_datasets \
    --reinit_pooler --normalize --mixout_layers 12 --mixout .3 \
    --trials 10''' > sample_commands/run.sh

In [41]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/leedtan/ModernML_TinyBert
   8e6771e..3524de5  some_frozen_mixout_reg_scaling -> origin/some_frozen_mixout_reg_scaling
Updating 8e6771e..3524de5
Fast-forward
 revisit-bert-finetuning/options.py | 4 ++--
 revisit-bert-finetuning/train.py   | 2 +-
 2 files changed, 3 insertions(+), 3 deletions(-)


In [ ]:
!sh sample_commands/run.sh

2020-12-21 17:01:56.501076: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/21/2020 17:01:58 - WARNING - run_glue -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
12/21/2020 17:01:59 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /root/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
12/21/2020 17:01:59 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "fine